# CRIM Intervals:  Modules

### What You Can Do with this Notebook

* Find contrapuntal modules, which are ngrams representing the combination of melodic and harmonic intervals made by every pair of voices in a piece
* Count and filter these modules
* Search for them in one piece or an entire corpus.

Read more about modules here:

https://github.com/HCDigitalScholarship/intervals/blob/rich_dev_22/tutorial/05_Ngrams.md#default-contrapuntal-module-ngrams



### A. Import Intervals and Other Code


In [2]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os


MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)
else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)
else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


## B. Importing a Piece

### B.1 Import a Piece and Check Title

In [3]:
# Select a prefix:

# prefix = 'Music_Files/'
prefix = 'https://crimproject.org/mei/'

# Add your filename here
mei_file = 'CRIM_Model_0032.mei'

# and combine the strings and load the piece
url = prefix + mei_file
piece = importScore(url)

print(piece.metadata)

{'title': 'Sancta et immaculata virginitas', 'composer': 'Cristóbal de Morales', 'date': 1546}


### C.1 Find Contrapuntal Modules

By default, `piece.ngrams()` will find ngrams of five elements:  three harmonic events surrounding two melodic ones in the lower voice of each pair.

But there are many other paramaters to adjust:

-  To set **length** as 5:  `piece.ngrams(n=5)`
    
-  To adjust **interval type** (d, c, z, or q): `piece.ngrams(interval_settings='d')` 
    
-  To determine the **'endpoint' of the offset reference** (as the start or end of the ngram): `piece.ngrams(offsets=
first)` or `piece.ngrams(offsets=last)`

- Return the melodic intervals for **both** voices (this can be helpful for finding cadences):  `piece.ngrams(show_both=True)`

Typical request:

`piece.ngrams(interval_settings='d', offsets='first', n=5).fillna('')`

Other advanced settings are noted in the documentation

To save an inventory of contrapuntal modules:  

`piece.ngrams(interval_settings='d', offsets='first', n=5).to_csv("saved_csv/mass_19_1_modules.csv")`

In [4]:
modules = piece.ngrams(interval_settings='d', 
                       offsets='first', 
                       n=3).fillna('')
modules.head(50)

,Bassus_Tenor,Bassus_Altus,Bassus_Superius,Tenor_Altus,Tenor_Superius,Altus_Superius
16.0,"5_Held, 5_Held, 5",,,,,
18.0,"5_Held, 5_Held, 3",,,,,
20.0,"5_Held, 3_-5, 10",,,,,
22.0,"3_-5, 10_5, 5",,,,,
24.0,"10_5, 5_Held, 4",,,,,
28.0,"5_Held, 4_Held, 5",,,,,
31.0,"4_Held, 5_1, 6",,,,,
32.0,"5_1, 6_1, 5","8_1, 8_1, 8",,"4_2, 3_-2, 4",,
34.0,"6_1, 5_-3, 8","8_1, 8_-3, 10",,"3_-2, 4_2, 3",,
36.0,"5_-3, 8_4, 5","8_-3, 10_4, 3",,"4_2, 3_Held, -3",,


In [5]:
# filtering results for a selected list of offsets
modules = piece.ngrams(interval_settings='d', offsets='last', n=3).fillna('')
my_offsets = [1, 2, 3, 10, 15, 27]
filtered_modules = modules.iloc[my_offsets,]
piece.detailIndex(df=filtered_modules, offset=True, beat=True)


Bassus_Tenor Bassus_Altus    Bassus_Superius  \
Measure Beat Offset                                                      
3       4.0  22.0    5_Held, 5_Held, 3                                   
4       1.0  24.0     5_Held, 3_-5, 10                                   
        3.0  28.0        3_-5, 10_5, 5                                   
6       1.5  41.0       8_4, 5_Held, 4                                   
7       1.0  48.0    3_Held, 2_Held, 3                                   
9       2.0  66.0                                    5_-4, 12_Held, 12   

                           Tenor_Altus     Tenor_Superius  Altus_Superius  
Measure Beat Offset                                                        
3       4.0  22.0                                                          
4       1.0  24.0                                                          
        3.0  28.0                                                          
6       1.5  41.0    3_Held, -3_-2, -2                                     
7       1.0  48.0         6_-2, 7_2, 6                                     
9       2.0  66.0                       11_2, 10_Held, 10  8_4, 5_Held, 5

### Modular nGrams with Various Options

- Diatonic, Chromatic, With-Quality, etc
- No Unisons
- Various lengths
- Exclude rests

In [6]:
mel = piece.melodic(kind="c")
har = piece.harmonic(kind="c")
ngrams = piece.ngrams(df=har, other=mel, n=3).fillna('')
ngrams.head()

,Bassus_Tenor,Bassus_Altus,Bassus_Superius,Tenor_Altus,Tenor_Superius,Altus_Superius
16.0,"7_Held, 7_Held, 7",,,,,
18.0,"7_Held, 7_Held, 3",,,,,
20.0,"7_Held, 3_-7, 15",,,,,
22.0,"3_-7, 15_7, 7",,,,,
24.0,"15_7, 7_Held, 5",,,,,


In [7]:
nr_no_unisons = piece.notes(combineUnisons=True)
mel = piece.melodic(df=nr_no_unisons, kind="d")
har = piece.harmonic(kind="d", compound=False)
ngrams = piece.ngrams(exclude=['Rest'], df=har, other=mel).fillna('')
ngrams.head()

,Bassus_Tenor,Bassus_Altus,Bassus_Superius,Tenor_Altus,Tenor_Superius,Altus_Superius
16.0,"5_Held, 5_Held, 5",,,,,
18.0,"5_Held, 5_Held, 3",,,,,
20.0,"5_Held, 3_-5, 3",,,,,
22.0,"3_-5, 3_5, 5",,,,,
24.0,"3_5, 5_Held, 4",,,,,


### C.2 Count Contrapuntal Modules

* how many of each `nGram` in this piece?
* variables for `kind`, `compound` `length`
* it is also possible exclude nGrams with **rests**, as shown here.

In [8]:

piece.ngrams(exclude=['Rest']).stack().value_counts().to_frame()

,0
"6_-2, 7_Held, 6",22
"5_Held, 4_Held, 3",21
"3_-2, 3_-2, 3",20
"3_Held, 2_Held, 3",19
"7_Held, 6_-2, 8",18
...,...
"6_Held, 5_-3, 10",1
"3_1, 3_Held, 10",1
"7_2, 6_-3, 8",1
"13_Held, 12_Held, 11",1


### C.3  Search for Modules

It's possible to search for any module, but cadences are quite formulaic, and thus are especially discoverable with this method.  Here are a few of the typical combinations expressed as modules.  Copy and paste some of these into the interactive search below to see where they occur in your piece.

* Authentic/Phrygian cadence with suspension in **diatonic**: `7_Held, 6_-2, 8` or `2_-3, 3_2, 1`

* Authentic cadence with suspension in **chromatic**: `10_Held, 9_-2, 12`

* Authentic cadence with suspension in **with quality**:`m7_Held, M6_-2, P8`

* Phrygian cadence with suspension in **chromatic**: `11_Held, 9_-2, 12`

* Phrygian cadence with suspension in **with quality**: `M7_Held, M6_-2, P8`

* Plagal cadence (no suspension) is often **diatonic**:  `6_-2, 6_-2, 6` at same time we hear `5_2,-4, 5`

In [9]:
@interact
def get_modules(search_pattern="", kind=["d", "q", "c", "z"], compound=[True, False], length=[3, 4, 5, 6], endpoint=["last", "first"]):
    
    piece_mel = piece.melodic(kind=kind)
    piece_har = piece.harmonic(kind=kind, compound=compound)
    ngrams = piece.ngrams(df=piece_har, other=piece_mel, n=length, offsets=endpoint)
    filtered_ngrams = ngrams[ngrams.apply(lambda x: x.astype(str).str.contains(search_pattern).any(), axis=1)]#.copy()
    beats_measures_mod = piece.detailIndex(filtered_ngrams, offset=True)

    return beats_measures_mod.fillna("-").applymap(str).style.applymap(lambda x: "background: #ccebc5" if re.search(search_pattern, x) else "")



interactive(children=(Text(value='', description='search_pattern'), Dropdown(description='kind', options=('d',…